In [172]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

In [124]:
path = "./data_cleaned.csv"
data = pd.read_csv(path)

In [125]:
nominal_columns = ["type","district"]

In [126]:
dummy_df = pd.get_dummies(data[nominal_columns], drop_first=False) 
data_df = pd.concat([data, dummy_df], axis=1)
data_df = data_df.drop(nominal_columns, axis=1)

In [151]:
data

,type,district,timestamp,duration_hour
0,['สะพาน'],สาทร,2021-09-26 05:03:52.594898+00:00,6068.222133
1,['ความสะอาด'],ประเวศ,2021-12-18 14:50:52.437512+00:00,4264.762677
2,['ท่อระบายน้ำ'],ประเวศ,2021-12-22 10:15:33.294829+00:00,4322.942139
3,['ความสะอาด'],ประเวศ,2021-12-28 03:59:06.003252+00:00,4706.848976
4,['สะพาน'],สาทร,2022-01-16 11:08:54.141813+00:00,3788.966330
...,...,...,...,...
103697,['ทางเท้า'],ราษฎร์บูรณะ,2023-05-16 11:13:58.680742+00:00,3.145034
103698,['ทางเท้า'],ราษฎร์บูรณะ,2023-05-16 11:16:13.611590+00:00,3.082024
103699,['ทางเท้า'],ราษฎร์บูรณะ,2023-05-16 11:18:30.176726+00:00,3.024274
103700,['ทางเท้า'],คลองเตย,2023-05-16 11:18:43.661156+00:00,2.218895


In [127]:
y = data_df.pop('duration_hour')
X = data_df.drop("timestamp",axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

In [128]:
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.shape)

(82961, 74)
(20741, 74)
(20741,)
(82961,)


In [129]:
from sklearn.metrics import mean_squared_error


In [160]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [40, 80,150],
    'max_features': [5, 10],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [300,400,500]
}
model = GridSearchCV(estimator=RandomForestRegressor(),param_grid= param_grid,cv=3,n_jobs=-1,verbose= 2) 

In [161]:
model.fit(X_train, y_train)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
[CV] END bootstrap=True, max_depth=40, max_features=5, min_samples_leaf=3, min_samples_split=8, n_estimators=300; total time=  17.2s
[CV] END bootstrap=True, max_depth=40, max_features=5, min_samples_leaf=3, min_samples_split=8, n_estimators=300; total time=  17.5s
[CV] END bootstrap=True, max_depth=40, max_features=5, min_samples_leaf=3, min_samples_split=8, n_estimators=300; total time=  17.5s
[CV] END bootstrap=True, max_depth=40, max_features=5, min_samples_leaf=3, min_samples_split=8, n_estimators=400; total time=  23.3s
[CV] END bootstrap=True, max_depth=40, max_features=5, min_samples_leaf=3, min_samples_split=8, n_estimators=400; total time=  23.3s
[CV] END bootstrap=True, max_depth=40, max_features=5, min_samples_leaf=3, min_samples_split=8, n_estimators=400; total time=  23.4s
[CV] END bootstrap=True, max_depth=40, max_features=5, min_samples_leaf=3, min_samples_split=8, n_estimators=500; total time=  28.5s
[CV] E

GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [40, 80, 150],
                         'max_features': [5, 10], 'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [300, 400, 500]},
             verbose=2)

In [162]:
print('Best param is')
print(model.best_params_)

Best param is
{'bootstrap': True, 'max_depth': 40, 'max_features': 10, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 300}


In [139]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [163]:
predictions = model.predict(X_test)

In [164]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

MAE: 1005.2417324410713
MSE: 2107775.5543475086
RMSE: 1451.8180169523687


In [166]:
print(predictions[-20:])

[ 873.60784748  317.54401839 1106.05086116  962.32515138 1024.98873286
  843.61854129 1373.54431737 1783.66310089  930.74635159 2204.99682196
  214.30164067  407.7990966  1860.22652359 2606.79097203  813.77490752
  407.7990966   114.58397588 1285.22954145  718.17462249  774.98772706]


In [140]:
best_grid = model.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)

Model Performance
Average Error: 1161.3784 degrees.
Accuracy = -10004.60%.


In [141]:
print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))


NameError: name 'base_accuracy' is not defined

In [169]:
lm = LinearRegression()
lm.fit(X_train,y_train)
print(lm)
print(lm.intercept_)
print(lm.n_features_in_)
print(lm.feature_names_in_)

predictionslm = lm.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, predictionslm))
print('MSE:', metrics.mean_squared_error(y_test, predictionslm))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictionslm)))

LinearRegression()
57793933640774.35
74
["type_['PM2.5']" "type_['การเดินทาง']" "type_['กีดขวาง']"
 "type_['คนจรจัด']" "type_['คลอง']" "type_['ความปลอดภัย']"
 "type_['ความสะอาด']" "type_['จราจร']" "type_['ต้นไม้']" "type_['ถนน']"
 "type_['ทางเท้า']" "type_['ท่อระบายน้ำ']" "type_['น้ำท่วม']"
 "type_['ป้าย']" "type_['ป้ายจราจร']" "type_['ร้องเรียน']"
 "type_['สอบถาม']" "type_['สะพาน']" "type_['สัตว์จรจัด']" "type_['สายไฟ']"
 "type_['ห้องน้ำ']" "type_['เสนอแนะ']" "type_['เสียงรบกวน']"
 "type_['แสงสว่าง']" 'district_คลองสาน' 'district_คลองสามวา'
 'district_คลองเตย' 'district_คันนายาว' 'district_จตุจักร'
 'district_จอมทอง' 'district_ดอนเมือง' 'district_ดินแดง' 'district_ดุสิต'
 'district_ตลิ่งชัน' 'district_ทวีวัฒนา' 'district_ทุ่งครุ'
 'district_ธนบุรี' 'district_บางกอกน้อย' 'district_บางกอกใหญ่'
 'district_บางกะปิ' 'district_บางขุนเทียน' 'district_บางคอแหลม'
 'district_บางซื่อ' 'district_บางนา' 'district_บางบอน' 'district_บางพลัด'
 'district_บางรัก' 'district_บางเขน' 'district_บางแค' 'dis

In [170]:
print(predictionslm[-20:])

[ 901.5546875   60.2578125 1060.1328125 1296.7734375  962.1328125
  931.140625  1231.8125    1893.0390625  928.0546875 1602.5078125
  144.515625   520.234375  1841.6640625 2114.7109375  965.7265625
  520.234375    89.15625   1092.1015625  665.2421875  872.2109375]


In [173]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictionslg = logmodel.predict(X_test)

ValueError: Unknown label type: 'continuous'